In [1]:
import hm
import pandas as pd


@@@@ This is HornMorpho, version 4.0.5 @@@@



In [2]:
df=pd.read_csv('C:/Users/Kois/LDA/36Tokenizedsentences.csv')
df=df.drop(['Unnamed: 0'], axis = 1)
df=df.to_numpy().tolist()

new_list=[]
temp_list=[]
for x in df:
    for item in x:
        if str(item) != 'nan':
            temp_list.append(item)
    new_list.append(temp_list)
    temp_list=[]
df=new_list

listOfNouns=[]
def nounDoc(l):
    Nouns=[]
    for x in l:
        y=hm.anal('amh',x,um=True)
        try:
            if y[1]['POS']=='n':
                Nouns.append(x)
        except:
            if y==[]:
                print()
            elif y[0]['POS']=='n':
               # print(x)
                Nouns.append(x)
    return Nouns

for item in df:
    listOfNouns.append(nounDoc(item))
 
listOfNouns

In [3]:
def getVerbs(l):
    verbs=[]
    for x in l:
        for item in x:
            if item not in verbs:
                try:
                    y=hm.anal('amh',item,um=True)
                    if y[0]['POS']=='v':
                        verbs.append(item)
                        f=open("verbs_corpus_36.txt","a",encoding="utf-8")
                        f.write((str(item)+","))
                        f.close()
                    elif y==[]:
                        print()
                except:
                    continue
    return verbs

verbs=getVerbs(df)

Loading FSTs for አማርኛ (analysis/generation) ...


In [4]:
verbs

['ጠራ',
 'ተናገረ',
 'አነበበ',
 'ገመተ',
 'ፈላ',
 'ኖረ',
 'ሰከረ',
 'መጣ',
 'ሲያቃጥልብህ',
 'መልሰህ',
 'አለቀለቀ',
 'ፈለገ',
 'ታገለ',
 'አጎነበሰ',
 'ቻለ',
 'ተጠቀመ',
 'አረገ',
 'አበረከተ',
 'ተረባረበ',
 'አወራ',
 'አዋለ',
 'ጠፋ',
 'ቀጠለ',
 'ተፈጸመ',
 'እስክል',
 'ጨረሰ',
 'ተሳካ',
 'ጠበቀ',
 'ፈታ',
 'አብሮ',
 'ቆመ',
 'አርጋ',
 'ተጫወተ',
 'ደከመ',
 'ሰራ',
 'አደነቀ',
 'አጠቃ',
 'ተለየ',
 'ይቅር',
 'ተማረ',
 'ገደለ',
 'ተፈጠረ',
 'ወለደ',
 'ታፈነ',
 'ተመለከተ',
 'ወከለ',
 'አማተበ',
 'ተከላከለ',
 'ረገመ',
 'ረካ',
 'ፈጸመ',
 'አቃጠለ',
 'አወገዘ',
 'አቀረበ',
 'ታወረ',
 'ጀመረ',
 'ተሻገረ',
 'አየ',
 'አሸነፈ',
 'ተፈታ',
 'ፈጽሞ',
 'ደገመ',
 'ፈረመ',
 'ተቃወመ',
 'ተጋ',
 'ያዘ']

In [5]:
no_verb=[]
verb_set=set(verbs)

for item in df:
    item_set=set(item)
    no_verb.append(list(item_set.difference(verb_set)))

In [9]:
clean_doc=no_verb
#clean_doc

In [10]:
import gensim
import numpy as np
from gsdmm import MovieGroupProcess

# cast tweets to numpy array
docs = clean_doc
#docs = processed_lemma
# create dictionary of all words in all documents
# initialize a Dictionary
dictionary = gensim.corpora.Dictionary(docs)

# filter extreme cases out of dictionary
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# create variable containing length of dictionary/vocab
vocab_length = len(dictionary)

# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

# initialize GSDMM
gsdmm = MovieGroupProcess(K=10, alpha=0.3, beta=0.9, n_iters=9)

# fit GSDMM model
y = gsdmm.fit(docs, vocab_length)

In stage 0: transferred 27 clusters with 8 clusters populated
In stage 1: transferred 27 clusters with 7 clusters populated
In stage 2: transferred 28 clusters with 9 clusters populated
In stage 3: transferred 30 clusters with 8 clusters populated
In stage 4: transferred 24 clusters with 6 clusters populated
In stage 5: transferred 32 clusters with 7 clusters populated
In stage 6: transferred 30 clusters with 7 clusters populated
In stage 7: transferred 27 clusters with 8 clusters populated
In stage 8: transferred 26 clusters with 7 clusters populated


In [12]:
word_cluster=[]
cluster_index=[]
# print number of documents per topic
doc_count = np.array(gsdmm.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('Most important clusters (by number of docs inside):', top_index)

# define function to get top words per topic
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        word_cluster.append(sort_dicts)
        cluster_index.append(cluster)
        print("\nCluster %s : %s"%(cluster, sort_dicts))

# get top words in topics
top_words(gsdmm.cluster_word_distribution, top_index, 3)

Number of documents per topic : [ 4  5  1  0  0  4  7  4 11  0]
Most important clusters (by number of docs inside): [8 6 1 7 5 0 2 9 4 3]

Cluster 8 : [('አሜን', 2), ('ደረሰ', 2), ('ሙስሊም', 2)]

Cluster 6 : [('መንግስት', 2), ('ተማሪ', 2), ('ታገተ', 2)]

Cluster 1 : [('ጨርሰ', 1), ('ህዋህት', 1), ('መከራ', 1)]

Cluster 7 : [('ብዙ', 1), ('አሸናፊ', 1), ('ቀንድ', 1)]

Cluster 5 : [('አብይ', 2), ('ሀገር', 1), ('ድርግ', 1)]

Cluster 0 : [('ፓርቲ', 2), ('ብልጽግና', 2), ('ጠላ', 2)]

Cluster 2 : [('ጥግ', 1), ('ራስ', 1), ('ሰማ', 1)]

Cluster 9 : []

Cluster 4 : []

Cluster 3 : []


In [13]:
# import library from gensim  
from gensim.models import CoherenceModel

# define function to get words in topics
def get_topics_lists(model, top_clusters, n_words):
    '''
    Gets lists of words in topics as a list of lists.
    
    model: gsdmm instance
    top_clusters:  numpy array containing indices of top_clusters
    n_words: top n number of words to include
    
    '''
    # create empty list to contain topics
    topics = []
    
    # iterate over top n clusters
    for cluster in top_clusters:
        #create sorted dictionary of word distributions
        sorted_dict = sorted(model.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:n_words]
        if len(sorted_dict)!=0:
            #create empty list to contain words
            topic = []

            #iterate over top n words in topic
            for k,v in sorted_dict:
                #append words to topic list
                topic.append(k)

            #append topics to topics list    
            topics.append(topic)
    
    return topics

# get topics to feed to coherence model
topics = get_topics_lists(gsdmm, top_index, 50) 

# evaluate model using Topic Coherence score
cm_gsdmm = CoherenceModel(topics=topics, 
                          model=gsdmm,
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs, 
                          coherence='c_v')

# get coherence value
coherence_gsdmm = cm_gsdmm.get_coherence()  

print(coherence_gsdmm)

0.5016052128227412


In [ ]:
def getVerbs(l):
    verbs=[]
    for x in l:
        for item in x:
            try:
                y=hm.anal('amh',item,um=True)
                if y[0]['POS']=='v':
                    verbs.append(item)
                    f=open("verbs_corpus.txt","a",encoding="utf-8")
                    f.write((str(item)+","))
                    f.close()
                elif y==[]:
                    print()
            except:
                continue
    return verbs

k=getVerbs(df)


for item in k:
    f=open("verbs_corpus.txt","a",encoding="utf-8")
    f.write((str(item)+","))
    f.close()